In [1]:
# ==========================================
# File: project_cdft/notebooks/03_final_training_xgb.ipynb
# ==========================================
# [CDFT-only] Final Training + Predictions (Ultimate Full Version)
#
# 核心功能:
# 1. 读取 01 生成的 Train/Val/Test 数据
# 2. 读取 02 生成的 best_params_xgb.json (智能清洗 regressor__xgb__ 前缀)
# 3. 训练 XGBoost (兼容 sklearn 接口，解决 early_stopping_rounds 报错)
# 4. 生成三份预测结果 (train/val/final) 供 04 绘图
# 5. 输出特征重要性 (Feature Importance)

import os
import json
import warnings
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

# ----------------------------------------------------------
# 0) 环境与路径准备
# ----------------------------------------------------------
# 确保在项目根目录下运行
if 'notebooks' in os.getcwd():
    os.chdir('..')
print(f"🚀 [03] 启动 XGB 训练 (CDFT 完整版) | 当前目录: {os.getcwd()}")

# ----------------------------------------------------------
# 1) 数据读取 (Data Loading)
# ----------------------------------------------------------
print("\n>>> [Step 1] Loading Data...")
try:
    df_train = pd.read_csv('data/processed/train.csv')
    df_val   = pd.read_csv('data/processed/val.csv')
    df_test  = pd.read_csv('data/processed/test.csv')
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到数据文件！请先运行 01_feature_engineering.ipynb")

# 自动识别 Target 列
target_col = 'Target_Log1o2' if 'Target_Log1o2' in df_train.columns else 'Target'

# 定义数据提取函数
def get_Xy(df: pd.DataFrame):
    y = df[target_col]
    # 排除非特征列 (Target, SMILES, ID等)
    X = df.drop(columns=[target_col, 'SMILES_Meta', 'SMILES', 'ID'], errors='ignore')
    # 只保留数值型特征
    X = X.select_dtypes(include=[np.number])
    return X, y

# 提取 X 和 y
X_train, y_train = get_Xy(df_train)
X_val,   y_val   = get_Xy(df_val)
X_test,  y_test  = get_Xy(df_test)

feature_names = list(X_train.columns)
print(f"   Train: {X_train.shape} | Val: {X_val.shape} | Test: {X_test.shape}")
print(f"   特征数量: {len(feature_names)}")

# ----------------------------------------------------------
# 2) 参数读取与清洗 (Params Loading)
# ----------------------------------------------------------
print("\n>>> [Step 2] Loading Parameters...")
PARAM_FILE = 'results/models/best_params_xgb.json'
params = {}

# 尝试读取 02 号文件生成的参数
if os.path.exists(PARAM_FILE):
    try:
        raw = json.load(open(PARAM_FILE, 'r', encoding='utf-8'))
        print(f"   ✅ 成功读取 02 号参数文件 ({len(raw)} 个参数)")

        # ★★★ 智能清洗前缀 ★★★
        # 02号文件可能带有 regressor__xgb__ 或 xgb__ 前缀，必须去掉才能给 XGBRegressor 用
        for k, v in raw.items():
            # 无论前缀是 regressor__xgb__ 还是 xgb__，取最后一部分作为参数名
            clean_key = k.split('xgb__')[-1]
            params[clean_key] = v

    except Exception as e:
        print(f"   ⚠️ 参数文件读取失败: {e}，将使用默认参数。")
else:
    print("   ⚠️ 未找到参数文件，将使用默认参数。")

# 设置默认参数 (CDFT 小样本稳健配置)
# 如果 params 里已经有了(来自02)，setdefault 不会覆盖它
defaults = {
    'objective': 'reg:squarederror',
    'n_jobs': -1,
    'random_state': 42,

    # 基础结构
    'max_depth': 3,            # 浅树防过拟合
    'learning_rate': 0.05,     # 慢工出细活
    'n_estimators': 5000,      # 给足空间，靠早停控制

    # 抗噪机制
    'subsample': 0.75,         # 行采样
    'colsample_bytree': 0.8,   # 列采样
    'reg_alpha': 0.1,          # L1 正则
    'reg_lambda': 1.0          # L2 正则
}

for k, v in defaults.items():
    params.setdefault(k, v)

# ★★★ 终极兼容性修复 ★★★
# 将 early_stopping_rounds 和 eval_metric 放入初始化参数中
# 这样可以避免在 fit() 中传参导致的 TypeError
params['early_stopping_rounds'] = 100
params['eval_metric'] = 'rmse'

print(f"\n🧠 最终训练参数:")
print(f"   LR: {params['learning_rate']} | Depth: {params['max_depth']} | EarlyStop: {params['early_stopping_rounds']}")

# ----------------------------------------------------------
# 3) 模型训练 (Training)
# ----------------------------------------------------------
print("\n>>> [Step 3] Training Model...")

# 初始化模型
model = XGBRegressor(**params)

# 训练 (fit 中只传数据和 eval_set，不传 callbacks)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=500  # 每500轮打印一次进度
)

# 获取最佳迭代轮数 (兼容不同版本)
best_iter = getattr(model, "best_iteration", params['n_estimators'])
print(f"\n✅ 训练完成! 最佳迭代轮数: {best_iter}")

# ----------------------------------------------------------
# 4) 预测与评估 (Prediction & Evaluation)
# ----------------------------------------------------------
print("\n>>> [Step 4] Predicting & Evaluating...")

def get_metrics(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return r2, rmse

# 预测
y_pred_train = model.predict(X_train)
y_pred_val   = model.predict(X_val)
y_pred_test  = model.predict(X_test)

# 计算指标
r2_tr, rmse_tr = get_metrics("train", y_train, y_pred_train)
r2_va, rmse_va = get_metrics("val",   y_val,   y_pred_val)
r2_te, rmse_te = get_metrics("test",  y_test,  y_pred_test)

print("\n" + "="*40)
print("📊 Performance Report (CDFT)")
print(f"   Train: R2={r2_tr:.4f} | RMSE={rmse_tr:.4f}")
print(f"   Val  : R2={r2_va:.4f} | RMSE={rmse_va:.4f}")
print(f"   Test : R2={r2_te:.4f} | RMSE={rmse_te:.4f}")
print("="*40)

# 简单诊断
if r2_tr - r2_va > 0.15:
    print("⚠️  诊断: 可能存在过拟合 (Train >> Val)，建议增加正则化或减少树深。")
elif r2_va < 0.5:
    print("⚠️  诊断: 可能存在欠拟合 (Val R2 低)，建议检查特征质量或降低学习率。")
else:
    print("✨ 诊断: 模型平衡性良好！")

# ----------------------------------------------------------
# 5) 结果保存 (Saving)
# ----------------------------------------------------------
print("\n>>> [Step 5] Saving Results...")
os.makedirs('results/predictions', exist_ok=True)

# 保存预测结果 (04号文件需要)
pd.DataFrame({target_col: y_train, 'Predicted': y_pred_train}).to_csv('results/predictions/train_predictions.csv', index=False)
pd.DataFrame({target_col: y_val,   'Predicted': y_pred_val}).to_csv('results/predictions/val_predictions.csv', index=False)
pd.DataFrame({target_col: y_test,  'Predicted': y_pred_test}).to_csv('results/predictions/final_predictions.csv', index=False)
print("💾 预测结果已保存至 results/predictions/ (Train/Val/Final)")

# 保存特征重要性 (Feature Importance)
try:
    os.makedirs('results/analysis', exist_ok=True)
    fi_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    fi_path = 'results/analysis/feature_importance_xgb.csv'
    fi_df.to_csv(fi_path, index=False)
    print(f"💾 特征重要性已保存至 {fi_path}")

    print("\n🧪 Top 5 最重要特征 (Top Drivers):")
    print(fi_df.head(5).to_string(index=False))
except Exception as e:
    print(f"⚠️ 特征重要性保存失败: {e}")

print("\n✅ 所有步骤完成！请运行 04_visualization.ipynb 查看图表。")

🚀 [03] 启动 XGB 训练 (CDFT 完整版) | 当前目录: D:\HO\my_smiles_project\y_g.baby\project_cdft

>>> [Step 1] Loading Data...
   Train: (105, 23) | Val: (23, 23) | Test: (23, 23)
   特征数量: 23

>>> [Step 2] Loading Parameters...
   ✅ 成功读取 02 号参数文件 (7 个参数)

🧠 最终训练参数:
   LR: 0.021371153508685993 | Depth: 2 | EarlyStop: 100

>>> [Step 3] Training Model...
[0]	validation_0-rmse:1.58265	validation_1-rmse:1.79010


[85]	validation_0-rmse:0.92179	validation_1-rmse:1.26949



✅ 训练完成! 最佳迭代轮数: 78

>>> [Step 4] Predicting & Evaluating...

📊 Performance Report (CDFT)
   Train: R2=0.6531 | RMSE=0.9424
   Val  : R2=0.4894 | RMSE=1.2632
   Test : R2=0.5121 | RMSE=1.2929
⚠️  诊断: 可能存在过拟合 (Train >> Val)，建议增加正则化或减少树深。

>>> [Step 5] Saving Results...
💾 预测结果已保存至 results/predictions/ (Train/Val/Final)
💾 特征重要性已保存至 results/analysis/feature_importance_xgb.csv

🧪 Top 5 最重要特征 (Top Drivers):
                   Feature  Importance
          Vertical IP (eV)    0.205784
Nucleophilicity index (eV)    0.154000
             Hardness (eV)    0.089728
            E-HOMO(N) (eV)    0.075823
          E-HOMO(N+1) (eV)    0.048068

✅ 所有步骤完成！请运行 04_visualization.ipynb 查看图表。
